In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import re
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from oauth2client import file, client, tools
import os
import httplib2

ModuleNotFoundError: No module named 'gspread'

In [9]:
# Run a new search, and extract the corresponding search id
search_string = 'https://search.larryhotz.com/search/results/?city=Denver&zip=all&neighborhood=all&type=res&type=con&search_status=SOLD&sold_price_min=all&sold_price_max=all&sold_within=1&short_sale=all&beds_min=all&baths_min=all&area_min=all&amenities=all&lot_size_min=all&year_built_min=all&levels=all&school_district=all&basement=all&sort_latest=true'
search_page = requests.get(search_string)
search_html1 = soup(search_page.content, 'html.parser')
search_html = search_html1.find('a', class_="savesearch_button")['href']
search_code = re.findall("/\w{4}/", search_html)[0]

search_code

'/4b8b/'

In [10]:
def clean_dat_string(value):
    value = value.text
    result = re.sub(r'[\t\r\n]', '', value).strip()
    return result

def parse_prop(prop_details, label):
    value = prop_details.find(text=label)
    if value is not None: 
        value = value.next_element.next_element.text
    return value

In [51]:
searchresult = requests.get(search_string)
searchresult
nprop = soup(searchresult.content,'html.parser').find('p',class_='results_properties_found').text
charn = re.findall(r'\d+',nprop)
n = int("".join(charn))
n

1227

In [84]:
# Initiates blank dataframe with the columns I want
#result_df = pd.DataFrame(columns=['sa1','sa2','sold_price','closing_date','prop_type','year_built','sq_ft','bedrooms','bathrooms','sub_div','sub_area','year_built','taxes','school_district','garage_spaces','parking_spaces','basement','hoa_fees','financial_terms'])
out_list=[]
# Loops through urls, scrapes the data, and saves it to the dataframe
for i in range(0,n):
    result_url = 'https://search.larryhotz.com/search/details'+search_code+str(i)
    result_page = requests.get(result_url)
    result_page = soup(result_page.content, 'html.parser')

    try:
        prop_details = result_page.findAll(class_='property_detail_specs')[0]
        prop_details2 = result_page.findAll(class_='property_detail_specs')[1]
        sa1 = clean_dat_string(result_page.find('h2'))
        sa2 = clean_dat_string(result_page.find('h3'))
    except Exception:
        pass
        
        
    if parse_prop(prop_details,'Closing Date') ==  'Mar 30, 2021':
        break
    outdict = {'sa1' : sa1,
                  'sa2' :sa2,
                  'closing_date':parse_prop(prop_details,'Closing Date'),
                'sold_price': parse_prop(prop_details,'List Price'),
        'sold_price': parse_prop(prop_details,'Sold Price'),
        'prop_type': parse_prop(prop_details,'Type'),
        'year_built': parse_prop(prop_details,'Year Built'),
        'sq_ft': parse_prop(prop_details,'Total Sqft'),
        'bedrooms': parse_prop(prop_details,'Bedrooms'),
        'bathrooms': parse_prop(prop_details,'Bathrooms'),
        'sub_div': parse_prop(prop_details,'Subdivision'),
        'sub_area': parse_prop(prop_details,'Sub-Area'),
        'year_built': parse_prop(prop_details,'Year Built'),
        'taxes': parse_prop(prop_details2,'Taxes'),
        'school_district': parse_prop(prop_details2,'School District'),
        'garage_spaces': parse_prop(prop_details2,'Garage Spaces'),
        'parking_spaces': parse_prop(prop_details2,'Parking Spaces'),
        'basement':parse_prop(prop_details2,'Basement'),
        'hoa_fees': parse_prop(prop_details2,'Total HOA Fees'),
        'financial_terms': parse_prop(prop_details2,'Financial Terms')}

    out_list.append(outdict)
    if (i % 10 ==0):
        print (i)

result_df = pd.DataFrame(out_list)       



0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160


KeyboardInterrupt: 

In [49]:
scopes = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_key_filename = 'denver-311215-14d0c79b4aa2.json'

credentials = Credentials.from_service_account_file(json_key_filename, scopes=scopes)
service = build('sheets', 'v4', credentials=credentials)
gc = gspread.authorize(credentials)


In [51]:
client_email = 'create-sales@denver-311215.iam.gserviceaccount.com'
spreadsheet_key = '1jRBmZigXTJHO5Y7HoJ6kRnyTSekZ4BS7je1lwfKqpRg'
wks_name = 'Sheet1'

sh1 = gc.open_by_key(spreadsheet_key).get_worksheet(0)
df_test = dataframe = pd.DataFrame(sh1.get_all_records())
df_test

set_with_dataframe(sh1, result_df)